In [1]:
# Import libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from ema_workbench.util import ema_logging, utilities


In [2]:
df_allscenarios = pd.read_excel('ModellAllData.xlsx')

In [3]:
# Change column names
df_allscenarios.rename(columns={'[run number]': 'Run_number', "Exits-available": "Exits_available","Familiarity-meter": "Familiarity", "initial-number-visitor": "Num_Visitors",
                       "vision-distance": 'Vision_distance', 'Chance-friends-in-building': "Chance_friends_in_building", 'vision-angle': "vision_angle", 
                        'gender-ratio': "gender_ratio", "chance-friends-nearby": "Chance_friends_nearby","Visitors-not-on-place": "Vistors_not_in_place",
                        'initial-number-staff': "Num_staff","[step]": "Step", 'total-evacuation-time': "Total_evacuation_time", '[ response-time ] of visitors': "Response_times_visitors",
                        '[ response-time ] of staff': "Response_times_staff", '[evacuation-time] of visitors': "Evac_times_visitors",
                        '[evacuation-time] of staff': 'Evac_times_staff', "[notif] of staff": "Notification_times_staff",
                        '[notif] of visitors': "Notification_times_visitors",'[response-time-excl-notif] of staff': "Response_times_staff_excNotif",
                        "[response-time-excl-notif] of visitors":"Response_times_visitors_excNotif", '[movement-time] of staff': "Movement_times_staff",
                        'gender-ratio': "gender_ratio", '[movement-time] of visitors':"Movement_times_visitors",'[response-tasks-list] of visitors':"Response_tasks_lists",
                'count staff with [ evacuated-state? =  True ]': "Staff_evacuated", 'count visitors with [ evacuated-state? =  True ]':"Visitors_evacuated", 
                               "count visitors with [informed-by-staff? = True]": "Visitors_informed_by_staff",
                               "count visitors with [fire-seen? = True]": "Visitors_fire_seen"}, inplace=True)

# Only keep useful columns
df_allscenarios= df_allscenarios[['Num_Visitors', 'Num_staff',  'Chance_friends_in_building', 'Familiarity',  'Culture',"Total_evacuation_time", 'Response_times_visitors',
 'Response_times_staff',  'Evac_times_visitors',  'Evac_times_staff',  'Notification_times_staff',  'Notification_times_visitors',  'Response_times_staff_excNotif',
 'Response_times_visitors_excNotif',  'Movement_times_staff',  'Movement_times_visitors',  'Response_tasks_lists', "Visitors_informed_by_staff", "Visitors_fire_seen"  ]].copy()

df_allscenarios=df_allscenarios.reset_index()

In [23]:
# Adjust lists columns; make them useable

Lists_columns= ['Response_times_visitors', 'Response_times_staff', 'Evac_times_visitors',  'Evac_times_staff', 'Notification_times_staff', 'Notification_times_visitors',
               'Response_times_staff_excNotif', 'Response_times_visitors_excNotif',  'Movement_times_staff',  'Movement_times_visitors' ]
for i in Lists_columns:
    df_allscenarios[i]=df_allscenarios[i].str.replace('[', '')
    df_allscenarios[i]=df_allscenarios[i].str.replace("]", "")
    df_allscenarios[i]=df_allscenarios[i].str.split(" ")

#Sort values in lists
for i in Lists_columns:
    df_allscenarios[i]=df_allscenarios[i].apply(lambda x: sorted(x))

# Transform strings to integers
for i in Lists_columns:
    for y in df_allscenarios.index:
        x= df_allscenarios.iloc[y][i]
        xx= list(map(int, x))
        df_allscenarios.at[y,i] = xx


In [24]:
#Add columns for evac 25, 50, 95, 100
df_allscenarios["Evac25"]= " "
df_allscenarios["Evac50"]= " "
df_allscenarios["Evac75"]= " "
df_allscenarios["Evac95"]= " "
df_allscenarios["AvgrespVist"]= " "
df_allscenarios["AvgrespStaff"]= " "
df_allscenarios["AvgevacVist"]= " "
df_allscenarios["AvgevacStaff"]= " "

# Add evac 25 , 50, 75 ,95, 100
for y in df_allscenarios.index:
    x= df_allscenarios.iloc[y]['Evac_times_visitors']
    a95= np.percentile(x,95)
    df_allscenarios["Evac95"][y] = a95
    a25= np.percentile(x,25)
    df_allscenarios["Evac25"][y] = a25
    a50= np.percentile(x,50)
    df_allscenarios["Evac50"][y] = a50
    a75= np.percentile(x,75)
    df_allscenarios["Evac75"][y] = a75
    
    z=df_allscenarios.iloc[y]['Response_times_visitors']
    df_allscenarios["AvgrespVist"][y]=np.mean(z)
    
    u=df_allscenarios.iloc[y]['Response_times_staff']
    df_allscenarios["AvgrespStaff"][y]=np.mean(u)
    
    o=df_allscenarios.iloc[y]['Evac_times_visitors']
    df_allscenarios["AvgevacVist"][y]=np.mean(o)
    
    g=df_allscenarios.iloc[y]['Evac_times_staff']
    df_allscenarios["AvgevacStaff"][y]=np.mean(g)


C:\Users\elvir\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\elvir\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\elvir\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\elvir\anaconda3\lib\site-packages\ipykernel_launch

In [25]:
# Add evacuated over time
df_allscenarios["Evac_over_time"]= " "
Evacuated_over_time_list= []
for y in df_allscenarios.index:
    Evacuated_over_time_list = []
    for i in range (2000):
        larger_elements = [element for element in df_allscenarios["Evac_times_visitors"][y] if element < i]
        number_of_elements = len(larger_elements)
        Evacuated_over_time_list.append(number_of_elements)
    df_allscenarios["Evac_over_time"][y]=Evacuated_over_time_list 


C:\Users\elvir\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [4]:
# Caculate percentages of people who have seen fire/ informed by staff
df_allscenarios["Visitors_fire_seen_percentage"]= " "
df_allscenarios["Visitors_informed_by_staff_percentage"]= " "
for y in df_allscenarios.index:
    x= df_allscenarios.iloc[y]['Visitors_fire_seen']
    df_allscenarios["Visitors_fire_seen_percentage"][y] = x/ (df_allscenarios.iloc[y]['Num_Visitors']) *100
    x= df_allscenarios.iloc[y]['Visitors_informed_by_staff']
    df_allscenarios["Visitors_informed_by_staff_percentage"][y] = x/ (df_allscenarios.iloc[y]['Num_Visitors']) *100

C:\Users\elvir\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\elvir\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [28]:
# create excel writer object
writer = pd.ExcelWriter('AllScenariosPreped.xlsx')
# write dataframe to excel
df_allscenarios.to_excel(writer)
# save the excel
writer.save()